### Generating and analysing pulses

This notebook shows how to generate custom shaped pulses, record them, and perform basic pulse shape analysis.

Import required libraries.

In [ ]:
from WF_SDK import device, scope, wavegen
import numpy as np
import matplotlib.pyplot as plt

Open device.

In [ ]:
device_data = device.open()

Create custom pulse shape.

In [ ]:
def gauss(x, A, x0, sigma):
    return A * np.exp(-(x - x0) ** 2 / (2 * sigma ** 2))

pulse = gauss(np.arange(-1000, 1000, 0.1), 1, 0, 100)

plt.plot(pulse)
plt.show()

Start generating pulses.

In [ ]:
wavegen.generate(device_data, channel=1, function=wavegen.function.custom, frequency=1e5, amplitude=1, offset=0, data=pulse)

Record data.

In [ ]:
scope.open(device_data, sampling_frequency=100e6, buffer_size=1000)
scope.trigger(device_data, enable=True, source=scope.trigger_source.analog, channel=1, edge_rising=True, level=0.1)
buffer = scope.record(device_data, channel=1)

Plot data.

In [ ]:
plt.plot(buffer)
plt.show()

Measure base line and pulse height.

In [ ]:
base = np.average(buffer[200:400])
height = np.max(buffer[400:900])
print(base, height)

Measure 10000 pulse heights.

In [ ]:
values = []
for i in range(10000):
    buffer = scope.record(device_data, channel=1)
    base = np.average(buffer[200:400])
    height = np.max(buffer[400:900])
    values.append(int((height - base) * 2048))

Plot histogram.

In [ ]:
counts, bins = np.histogram(values, bins=range(2048))
plt.step(bins[2000:-1], counts[2000:], where="post")
plt.show()

Close device.

In [ ]:
device.close(device_data)